In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestClassifier

### Leitura dos dados

In [ ]:
df_train = pd.read_csv("../input/airline-passenger-satisfaction/train.csv")
df_test = pd.read_csv("../input/airline-passenger-satisfaction/test.csv")

df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]

df = df_train.append(df_test)

df.head(5)

## Análise descritiva

In [ ]:
df.describe()

In [ ]:
df.info()

### Verificando dados da variável resposta

In [ ]:
df_satisfaction = df.satisfaction.value_counts()

sns.barplot(x=df_satisfaction.index, y=df_satisfaction, palette='BuPu')

* A distribuição da variável resposta está balanceada

### Análise de satisfação

In [ ]:
def group_by_satisfaction(column, i):
    df_satisfaction = (
        pd.DataFrame(df.groupby([column, 'satisfaction'])['id'].count())
        .reset_index()
    )
    
    return df_satisfaction

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=1, figsize=(10,25))
cols = ["Class", "Customer Type", "Gender", "Type of Travel"]

for i, feat in enumerate(cols):
    df_satisfaction = group_by_satisfaction(feat, i)

    sns.barplot(
        data=df_satisfaction,
        x=feat, y="id", hue="satisfaction",
        ci="sd", palette="BuPu", ax=ax[i]
    )

### Satisfação por idade e tempo de vôo

In [ ]:
f1, ax = plt.subplots(nrows=4, ncols=1, figsize=(10,15))
cols = ['Age', 'Flight Distance']
    
sns.histplot(df, x=cols[0], bins=25, palette='BuPu', hue="satisfaction", ax=ax[0])
sns.boxplot(y='satisfaction', x=cols[0], data=df, orient='h', width=0.5, palette='BuPu', ax=ax[1])

sns.histplot(df, x=cols[1], bins=25, palette='BuPu', hue="satisfaction", ax=ax[2])
sns.boxplot(y='satisfaction', x=cols[1], data=df, orient='h', width=0.5, palette='BuPu', ax=ax[3])

* Pessoas mais velhas estão mais satisfeitas do que as mais jovens
* Vôos curtos possuem uma insatisfação maior do que vôos longos

### Análise da qualidade dos serviços

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,8))

columns = df.loc[:,['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']]

sns.boxplot(x='value', y='variable', data = columns.melt(), palette='BuPu', orient='h')

* A maioria dos serviços possui mediana = 3

### Correlação entre as variáveis

In [ ]:
df_cor = df.drop('id',axis=1)

plt.figure(figsize=(25, 10))
mask = np.triu(np.ones_like(df_cor.corr(method='spearman'), dtype=np.bool))
heatmap = sns.heatmap(df_cor.corr(method='spearman'), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BuPu')

## Random Forest

### Transformando variáveis categóricas

In [ ]:
def convert_categorical(data):
    data['Class'] = data['Class'].map({'Business': 2, 'Eco Plus': 1, 'Eco': 0})
    data['satisfaction'] = data['satisfaction'].map({'neutral or dissatisfied': 0, 'satisfied': 1})
    data['Type of Travel'] = data['Type of Travel'].map({'Personal Travel': 0, 'Business travel': 1})
    data['Customer Type'] = data['Customer Type'].map({'Loyal Customer': 1, 'disloyal Customer': 0})

    data = pd.get_dummies(data)
    data.drop('id',axis=1,inplace=True)
    data.fillna(0,inplace=True)

    return data

train = convert_categorical(df_train)
test = convert_categorical(df_test)

### Modelo

#### Preparação das variáveis do modelo

In [ ]:
removed_cols = ["id", "satisfaction"]

feats = [c for c in train.columns if c not in removed_cols]

x_test = test[feats]
y_test = test['satisfaction']

x_train = train[feats]
y_train = train['satisfaction']

#### Treinamento e predição do modelo

In [ ]:
random_forest = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=200)

random_forest.fit(x_train, y_train)

pred = random_forest.predict(x_test)

train_acc = random_forest.score(x_train, y_train) * 100
test_acc = random_forest.score(x_test, y_test) * 100

print(f'Acurácia de dados de treino: {round(train_acc, 2)}%')
print(f'Acurácia de dados de teste: {round(test_acc, 2)}%')

#### Matriz de confusão dos dados de teste

In [ ]:
cm = confusion_matrix(y_test, pred)

f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(cm, fmt=".0f", annot=True, linewidths=0.2, linecolor="purple", ax=ax, cmap='BuPu')
plt.xlabel("Valor Predito")
plt.ylabel("Valor Real")
plt.show()

In [ ]:
print("Classification Report")
print(metrics.classification_report(y_test, pred))

#### Curva Característica de Operação do Receptor
- A curva ROC é obtido pela representação da razão RPV = Positivos Verdadeiros / Positivos Totais versus a razão RPF = Positivos Falsos / Negativos Totais, para vários valores do limiar de classificação.

- A RPV define quantos resultados positivos corretos ocorrem entre as amostras positivas disponíveis durante o teste. A RPF,por outro lado, define quantos resultados positivos incorretos ocorrem entre todas as amostras negativas disponíveis durante o teste. 

- AUC (área embaixo da curva) fornece uma medida agregada de desempenho em todos os limites de classificação possíveis. Uma forma de interpretar AUC é como a probabilidade de que o modelo classifique um exemplo positivo aleatório mais alto do que um exemplo negativo aleatório.

In [ ]:
y_pred_proba = random_forest.predict_proba(x_test)[::,1]

fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.plot(fpr, tpr, label="data, auc="+str(auc))
plt.xlabel('Falso positivo')
plt.ylabel('Verdadeiro positivo')
plt.legend(loc=4)
plt.show()

### Avaliação do modelo

In [ ]:
mean_squared_error(y_test, pred)

In [ ]:
mean_squared_log_error(np.exp(y_test), np.exp(pred))**(1/2)

* O valor do erro foi baixo, o que significa que o modelo consegue explicar bem os dados